In [58]:
import pandas as pd
import numpy as np

data = pd.read_excel('300164.xls')
data

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
0,300164,通源石油,2011-01-13,9.008070,9.523183,8.798506,9.372808,——,——,10825154.0,626364272.0,0.0,0.0
1,300164,通源石油,2011-01-14,8.961679,8.987274,8.435368,8.435368,-0.937441,-10.00171,6344591.0,337741849.0,0.0,0.0
2,300164,通源石油,2011-01-17,8.139418,8.366579,7.920255,8.157015,-0.278353,-3.299829,3279010.0,165847606.0,0.0,0.0
3,300164,通源石油,2011-01-18,7.998641,7.998641,7.680295,7.837070,-0.319945,-3.922334,2287376.0,112105361.0,0.0,0.0
4,300164,通源石油,2011-01-19,7.837070,8.006640,7.819472,7.982645,0.145575,1.857519,1615372.0,79809744.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2792,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2793,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2794,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 寻找由字线和甲字线

In [2]:
def is_lower_hammer(data, lam=2):
    """
    由字线的判断：一侧wick长度是body长度两倍以上，另一侧wick长度显著更小
    :param data: 数据，包括OLHC
    :param O: 开盘价，numpy 1d array
    :param H: 最高价，numpy 1d array
    :param L: 最低价，numpy 1d array
    :param C: 收盘价，numpy 1d array
    :param lam: 一个控制严格程度的参数（蜡烛主体长度要大于较短影线的多少倍）
    :return: Boolean
    """
    O = np.array(data['开盘价'])
    L = np.array(data['最低价'])
    H = np.array(data['最高价'])
    C = np.array(data['收盘价'])
    body_size = np.abs(O - C)  # 开盘价与收盘价之间差的绝对值为蜡烛主体长度
    wick_size_upper = np.abs(H - np.maximum(O, C))  # 一根影线的长度，有H >= max(O, C)
    wick_size_lower = np.abs(np.minimum(O, C) - L)  # 另一根影线长度，同理
    return (2 * body_size < wick_size_upper) & (body_size > lam * wick_size_lower)

In [3]:
lower_hammer_mask = is_lower_hammer(data)

In [4]:
data[lower_hammer_mask]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
5,300313,*ST天山,2012-05-04,7.759188,8.360982,7.729592,7.941700,0.088789,1.130651,7346334,119100367,0,0
7,300313,*ST天山,2012-05-08,7.966363,8.494165,7.960937,8.129143,0.335425,4.303792,8243071,136944001,0,0
23,300313,*ST天山,2012-05-30,7.113000,7.369502,7.108068,7.157395,-0.004933,-0.068873,1641707,24116528,0,0
43,300313,*ST天山,2012-06-28,7.231386,7.458292,7.231386,7.275781,0,0,1517670,22618479,0,0
157,300313,*ST天山,2012-12-12,5.324883,5.558780,5.324883,5.399531,0.074648,1.401869,1073478,11738404,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,300313,*ST天山,2023-02-02,6.350000,6.580000,6.240000,6.240000,-0.06,-0.952388,6030936,38545101,0,0
2390,300313,*ST天山,2023-04-18,6.000000,6.300000,5.980000,6.070000,0.07,1.166669,6861718,41802251,0,0
2400,300313,*ST天山,2023-05-08,5.500000,5.800000,5.490000,5.590000,0.06,1.08499,3468900,19547975,0,0
2435,300313,*ST天山,2023-06-28,4.910000,5.080000,4.890000,4.960000,0.06,1.224489,3689083,18452422,0,0


In [5]:
def is_upper_hammer(data, lam=2):
    """
    甲字线的判断：一侧wick长度是body长度两倍以上，另一侧wick长度显著更小
    :param data: 数据，包括OLHC
    :param O: 开盘价，numpy 1d array
    :param H: 最高价，numpy 1d array
    :param L: 最低价，numpy 1d array
    :param C: 收盘价，numpy 1d array
    :param lam: 一个控制严格程度的参数（蜡烛主体长度要大于较短影线的多少倍）
    :return: Boolean
    """
    O = np.array(data['开盘价'])
    L = np.array(data['最低价'])
    H = np.array(data['最高价'])
    C = np.array(data['收盘价'])
    body_size = np.abs(O - C)  # 开盘价与收盘价之间差的绝对值为蜡烛主体长度
    wick_size_upper = np.abs(H - np.maximum(O, C))  # 一根影线的长度，有H >= max(O, C)
    wick_size_lower = np.abs(np.minimum(O, C) - L)  # 另一根影线长度，同理
    return (2 * body_size < wick_size_lower) & (body_size > lam * wick_size_upper)

In [6]:
upper_hammer_mask = is_upper_hammer(data)
data[upper_hammer_mask]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
6,300313,*ST天山,2012-05-07,7.680264,7.823314,7.408965,7.793717,-0.147982,-1.863356,5054014,78654417,0,0
76,300313,*ST天山,2012-08-14,6.027799,6.091925,5.869952,6.077127,0.029596,0.489395,1017155,12348524,0,0
117,300313,*ST天山,2012-10-17,5.623315,5.623315,5.519728,5.608517,0.014798,0.264541,397552,4495825,0,0
131,300313,*ST天山,2012-11-06,5.867324,5.867324,5.723005,5.822536,-0.069672,-1.182437,391542,4545571,0,0
132,300313,*ST天山,2012-11-07,5.812583,5.847418,5.747888,5.837465,0.014929,0.256405,323151,3770722,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,300313,*ST天山,2022-12-30,6.140000,6.160000,6.090000,6.160000,0.06,0.983606,2211300,13539028,0,0
2335,300313,*ST天山,2023-01-30,5.910000,6.030000,5.650000,6.020000,0.02,0.333333,5321218,31263544,800,4816
2440,300313,*ST天山,2023-07-05,5.010000,5.050000,4.940000,5.040000,-0.04,-0.787401,1960600,9793736,0,0
2442,300313,*ST天山,2023-07-07,4.950000,4.950000,4.830000,4.920000,-0.01,-0.202835,2021900,9910884,0,0


# 好友反攻

In [7]:
def long_bull_candle(stock_data, lam=3, factor=1):
    """
    判断是否是大阳线，标准：主体占大部分（主体大于 影线长度 * lam），其次判断是否足够长，标准：长度大于平均值+n个标准差
    :param stock_data: 数据，包括OLHC
    :param O: open numpy 1d array
    :param H: high numpy 1d array
    :param L: low numpy 1d array
    :param C: close numpy 1d array
    :param lam: controlling parameter
    :param data: the given dataframe
    :param factor: how many times of std is body_size supposed to excess
    :return: mask matrix numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    upper_wick_size = np.abs(H - np.maximum(O, C))
    lower_wick_size = np.abs(np.minimum(O, C) - L)
    data_feature_body = np.abs(stock_data['开盘价'] - stock_data['收盘价']) / stock_data['收盘价']
    return (body_size / C > data_feature_body.mean() + data_feature_body.std() * factor) & (body_size > lam * upper_wick_size) & (body_size > lower_wick_size) & (O < C)

In [8]:
def long_bear_candle(stock_data, lam=3, factor=1):
    """
    判断是否是大阳线，标准：主体占大部分（主体大于 影线长度 * lam），其次判断是否足够长，标准：长度大于平均值+n个标准差
    :param stock_data: 数据，包括OLHC
    :param O: open numpy 1d array
    :param H: high numpy 1d array
    :param L: low numpy 1d array
    :param C: close numpy 1d array
    :param lam: controlling parameter
    :param data: the given dataframe
    :param factor: how many times of std is body_size supposed to excess
    :return: mask matrix numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    upper_wick_size = np.abs(H - np.maximum(O, C))
    lower_wick_size = np.abs(np.minimum(O, C) - L)
    data_feature_body = np.abs(stock_data['开盘价'] - stock_data['收盘价']) / stock_data['收盘价']
    return (body_size / C > data_feature_body.mean() + data_feature_body.std() * factor) & (body_size > lam * upper_wick_size) & (body_size > lower_wick_size) & (O > C)

In [9]:
def is_friend_counter(stock_data, lam=1, factor=1):
    """
    判断是否为好友反攻
    :param stock_data: 股票数据，dataframe
    :param lam: controlling param
    :param factor: controlling param
    :return: mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    long_bear_candle_mask = long_bear_candle(data, lam=lam, factor=factor)
    return long_bull_candle_mask & np.roll(long_bear_candle_mask, 1) & (np.abs(C - np.roll(C, 1)) < ((1 / 3) * body_size))

In [10]:
data[is_friend_counter(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
1142,300313,*ST天山,2018-02-02,10.02,11.50,10.02,10.94,0.139999,1.296291,2832805,30783819,0,0
1825,300313,*ST天山,2020-12-16,22.80,25.17,22.22,24.05,0.269999,1.135402,33553284,807635984,1900,45695


In [11]:
def is_oldmam_counter(stock_data, lam=1, factor=1):
    """
    判断是否为淡友反攻
    :param stock_data: 股票数据，dataframe
    :param lam: controlling param
    :param factor: controlling param
    :return: mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    long_bear_candle_mask = long_bear_candle(data, lam=lam, factor=factor)
    return long_bear_candle_mask & np.roll(long_bull_candle_mask, 1) & (np.abs(C - np.roll(C, 1)) < ((1 / 3) * body_size))

In [12]:
data[is_oldmam_counter(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
789,300313,*ST天山,2016-04-15,22.35,22.65,20.26,21.05,-0.370001,-1.727362,19646278,415637888,0,0


In [13]:
def beam_of_light(stock_data, lam=1, factor=1):
    """
    判断是否为曙光初现
    :param stock_data: 股票数据，dataframe
    :param lam: controlling param
    :param factor: controlling param
    :return: mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    long_bear_candle_mask = long_bear_candle(data, lam=lam, factor=factor)
    return long_bull_candle_mask & np.roll(long_bear_candle_mask, 1) & (np.abs(C - np.roll(C, 1)) < ((1 / 2) * np.roll(body_size, 1)))

In [14]:
data[beam_of_light(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
745,300313,*ST天山,2015-08-19,16.47,18.03,15.50,18.00,0.780001,4.529621,5180538,86293240,0,0
1142,300313,*ST天山,2018-02-02,10.02,11.50,10.02,10.94,0.139999,1.296291,2832805,30783819,0,0
1825,300313,*ST天山,2020-12-16,22.80,25.17,22.22,24.05,0.269999,1.135402,33553284,807635984,1900,45695
1996,300313,*ST天山,2021-08-30,7.88,8.60,7.88,8.33,0.29,3.606965,13268300,110914420,0,0
2033,300313,*ST天山,2021-10-29,9.16,10.08,9.16,9.70,0.24,2.536995,7895255,76872581,0,0


In [17]:
def rising_sun(stock_data, lam=0.2):
    """
    判断是否为旭日东升
    :param stock_data: 股票数据 dataframe
    :param lam: 控制参数
    :return: boolean musk numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=1.5)
    long_bear_candle_mask = long_bear_candle(data, lam=lam, factor=1.5)
    return np.roll(long_bear_candle_mask, 1) & long_bull_candle_mask & (np.abs(C - np.roll(C, 1)) > lam * body_size)

In [18]:
data[rising_sun(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
745,300313,*ST天山,2015-08-19,16.47,18.03,15.50,18.00,0.780001,4.529621,5180538,86293240,0,0
763,300313,*ST天山,2015-09-16,13.30,14.78,13.20,14.70,1.26,9.375002,4513796,63224246,0,0
1442,300313,*ST天山,2019-05-07,5.15,5.63,5.15,5.63,0.51,9.960942,6660135,36191862,0,0
1452,300313,*ST天山,2019-05-21,5.01,5.50,5.01,5.50,0.5,10,2985897,15982287,0,0
1465,300313,*ST天山,2019-06-10,5.42,5.95,5.42,5.95,0.54,9.981515,15012175,86248165,0,0
1916,300313,*ST天山,2021-05-07,7.65,8.98,7.50,8.98,1.5,20.05347,27429616,229468691,1100,9878


In [19]:
def clouded_top(stock_data, lam=0.5, factor=1):
    """
    判断是否为乌云盖顶
    :param stock_data: 股票数据 dataframe
    :param lam: 控制参数
    :param factor: 控制参数
    :return: boolean mask 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    long_bear_candle_mask = long_bear_candle(data, lam=lam, factor=factor)
    return np.roll(long_bull_candle_mask, 1) & long_bear_candle_mask & ((np.roll(O, 1) - C) < lam * np.roll(body_size, 1))

In [21]:
data[clouded_top(data, factor=1.5)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
612,300313,*ST天山,2015-01-30,13.00655,13.46073,12.00336,12.0333,-0.973245,-7.482729,4221247,107319539,0,0
712,300313,*ST天山,2015-07-01,22.79000,23.29000,20.44000,20.4400,-2.269999,-9.99559,3976826,87676825,0,0
1792,300313,*ST天山,2020-10-30,24.28000,24.68000,21.13000,21.2000,-2.559999,-10.77441,38443900,862132912,4000,84800
1801,300313,*ST天山,2020-11-12,22.02000,22.02000,20.45000,20.5700,-1.59,-7.175091,20535100,427561872,4000,82280
1860,300313,*ST天山,2021-02-04,16.13000,16.15000,14.10000,14.7500,-1.549999,-9.509198,26922576,407833168,400,5900


In [22]:
def raining_cats(stock_data, lam=3, factor=1):
    """
    识别是否为倾盆大雨
    :param stock_data: 股票数据 dataframe
    :param lam: 控制因子
    :param factor: 控制因子
    :return: mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    long_bear_candle_mask = long_bear_candle(data, lam=lam, factor=factor)
    return np.roll(long_bull_candle_mask, 1) & long_bear_candle_mask & (C < np.roll(O, 1)) & (O < np.roll(C, 1))

In [25]:
data[(raining_cats(data))]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
563,300201,海伦哲,2013-08-07,1.994316,2.010935,1.898754,1.907064,-0.099716,-4.968941,9160646,42874630,0,0
1988,300201,海伦哲,2020-09-24,4.640000,4.730000,4.280000,4.340000,-0.41,-8.631576,55410000,247191693,8500,36890


In [50]:
def bullish_end_line(stock_data, lam=3, factor=1, delta=0.8):
    """
    判断是否是尽头线，标准：出现在涨势中，第一根Ｋ线为大阳线或中阳线，并留有一根上影线，第二根为小十字线或小阳小阴线，依附在第一根的上影线之内，是卖出信号。
    出现在跌势中，第一根Ｋ线为大阴线或中阴线，并留有一根下影线，第二根Ｋ线为小十字线或小阳小阴线，依附在第一根Ｋ线的下影线之内，是买进信号。
    此函数判断是否为看涨尽头线
    已放宽条件至，第一天只要是阳线就可以
    :param stock_data: 股票数据 dataframe
    :param lam: 控制参数
    :param factor: 控制参数
    :param delta: 控制参数
    :return: boolean mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    upper_wick_size = np.abs(H - np.maximum(O, C))
    lower_wick_size = np.abs(np.minimum(O, C) - L)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    first_day_condition = (upper_wick_size > delta * body_size) & (lower_wick_size > delta * body_size) & (C > O)
    return np.roll(first_day_condition, 1) & (H < np.roll(H, 1)) & (L > np.roll(C, 1))

In [59]:
data[bullish_end_line(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
233,300164,通源石油,2011-12-29,5.037732,5.135098,5.037732,5.082520,0.085682,1.714732,130221.0,3396479.0,0.0,0.0
1029,300164,通源石油,2016-01-06,8.792888,9.061080,8.773730,9.022767,0.28735,3.289479,8572711.0,79901939.0,0.0,0.0


In [60]:
def bearish_end_line(stock_data, lam=3, factor=1, delta=0.8):
    """
    判断是否是尽头线，标准：出现在涨势中，第一根Ｋ线为大阳线或中阳线，并留有一根上影线，第二根为小十字线或小阳小阴线，依附在第一根的上影线之内，是卖出信号。
    出现在跌势中，第一根Ｋ线为大阴线或中阴线，并留有一根下影线，第二根Ｋ线为小十字线或小阳小阴线，依附在第一根Ｋ线的下影线之内，是买进信号。
    此函数判断是否为看跌尽头线
    已放宽条件至，第一天只要是阴线就可以
    :param stock_data: 股票数据 dataframe
    :param lam: 控制参数
    :param factor: 控制参数
    :param delta: 控制参数
    :return: boolean mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    upper_wick_size = np.abs(H - np.maximum(O, C))
    lower_wick_size = np.abs(np.minimum(O, C) - L)
    long_bull_candle_mask = long_bull_candle(data, lam=lam, factor=factor)
    first_day_condition = (upper_wick_size > delta * body_size) & (lower_wick_size > delta * body_size) & (C < O)
    return np.roll(first_day_condition, 1) & (H < np.roll(C, 1)) & (L > np.roll(L, 1))

In [61]:
data[bearish_end_line(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
842,300164,通源石油,2014-11-12,11.9657,12.01357,11.82211,11.8604,-0.181878,-1.510329,5456614.0,67734242.0,0.0,0.0
2159,300164,通源石油,2021-03-10,3.8600,3.90000,3.72000,3.7500,-0.21,-5.303031,23410234.0,88883516.0,0.0,0.0
2317,300164,通源石油,2021-11-03,4.9000,5.01000,4.82000,4.9500,-0.1,-1.980206,29200277.0,143816172.0,0.0,0.0


In [62]:
def bear_line(stock_data):
    """
    判断是否是孕育线
    :param stock_data: 股票数据 dataframe
    :return: boolean mask numpy 1d array
    """
    O = np.array(stock_data['开盘价'])
    L = np.array(stock_data['最低价'])
    H = np.array(stock_data['最高价'])
    C = np.array(stock_data['收盘价'])
    body_size = np.abs(O - C)
    return (np.roll(np.maximum(O, C), 1) > np.maximum(O, C)) & (np.roll(np.minimum(O, C), 1) < np.minimum(O, C))

In [63]:
data[bear_line(data)]

,证券代码,证券名称,交易时间,开盘价,最高价,最低价,收盘价,涨跌,涨跌幅%,成交量,成交额,盘后量,盘后额
8,300164,通源石油,2011-01-25,7.328356,7.417940,7.251569,7.357151,0.030396,0.414856,645705.0,29612269.0,0.0,0.0
11,300164,通源石油,2011-01-28,7.608308,7.654700,7.493127,7.566715,-0.06399,-0.838581,611752.0,28921069.0,0.0,0.0
13,300164,通源石油,2011-02-01,7.937852,7.944251,7.790677,7.824271,-0.17597,-2.19956,730422.0,35862936.0,0.0,0.0
22,300164,通源石油,2011-02-21,8.142617,8.262597,8.142617,8.219404,0.087985,1.082038,854054.0,43789205.0,0.0,0.0
44,300164,通源石油,2011-03-23,8.099424,8.131419,8.024238,8.110622,0.049591,0.615195,205343.0,10375943.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,300164,通源石油,2023-05-19,4.530000,4.570000,4.440000,4.460000,-0.14,-3.043475,27569300.0,123311466.0,0.0,0.0
2696,300164,通源石油,2023-05-26,4.470000,4.550000,4.400000,4.490000,-0.02,-0.443469,18634698.0,83065569.0,0.0,0.0
2709,300164,通源石油,2023-06-14,4.410000,4.450000,4.370000,4.400000,0.02,0.45662,14326705.0,63048766.0,1500.0,6600.0
2730,300164,通源石油,2023-07-17,4.710000,4.710000,4.570000,4.680000,-0.09,-1.886796,15616500.0,72737320.0,0.0,0.0
